In [46]:
import numpy as np 
import pandas as np 
import torchvision 
import torch.nn as nn 
import torch
import torch.nn.functional as F 
import torch 
import os 
from torchvision import models 
from torchsummary import summary
import tqdm as notebook_tqdm
from torchvision.models.resnet import resnet50, resnet101, resnet152

run_experiment4 = True 

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device 

device(type='cuda')

# Lenet5 cap


In [12]:
# Conv NN constructors:
class ConvNN(nn.Module):
    name = ""

    def __init__(self, params):
        """
        The purpose in using this class in the way it built is to make the process of creating CNNs with the ability
        to control its capacity in efficient way (programming efficiency - not time efficiency).
        :param params: a dictionary with the following attributes:
            capacity influence:
            - channels_lst: lst of the channels sizes. the network complexity is inflected mostly by this parameter
              * for efficiency channels_lst[0] is the number of input channels
            - #FC_Layers: number of fully connected layers
            - extras_blocks_components: in case we want to add layers from the list ["dropout", "max_pool", "batch norm"]
                                        to each block we can do it. Their parameters are attributes of this dict also.
              * notice that if max_pool in extras_blocks_components then we reduce dims using max_pool instead conv
                layer (the conv layer will be with stride 1 and padding)
            - p_dropout: the dropout parameter

            net structure:
            - in_wh: input width and height
        """
        super().__init__()
        self.params = params
        channels_lst = params["channels_lst"]
        extras_block_components = params["extras_blocks_components"]

        assert 2 <= len(channels_lst) <= 5
        conv_layers = []
        for i in range(1, len(channels_lst)):
            """
            Dims calculations: next #channels x (nh-filter_size/2)+1 x (nw-filter_size/2)+1
            """
            filter_size, stride, padding = (4, 2, 1) if "max_pool" not in extras_block_components else (5, 1, 2)
            conv_layers.append(nn.Conv2d(channels_lst[i - 1], channels_lst[i], filter_size, stride, padding, bias=False))
            conv_layers.append(params["activation"]())

            for comp in extras_block_components:
                if comp == "dropout":
                    conv_layers.append(nn.Dropout(params["p_dropout"]))
                if comp == "max_pool":
                    conv_layers.append(nn.MaxPool2d(2, 2))
                if comp == "batch_norm":
                    conv_layers.append(nn.BatchNorm2d(channels_lst[i]))

        out_channels = channels_lst[-1]
        if params["CNN_out_channels"] is not None:
            conv_layers.append(nn.Conv2d(channels_lst[-1], params["CNN_out_channels"], 1))
            conv_layers.append(params["activation"]())
            out_channels = params["CNN_out_channels"]

        self.cnn = nn.Sequential(*conv_layers)

        lin_layers = []
        wh = params["in_wh"] // (2 ** (len(channels_lst) - 1))  # width and height of last layer output
        lin_layer_width = out_channels * (wh ** 2)
        for _ in range(params["#FC_Layers"] - 1):
            lin_layers.append(nn.Linear(lin_layer_width, lin_layer_width))
        lin_layers.append(nn.Linear(lin_layer_width, params["out_size"]))
        self.linear_nn = nn.Sequential(*lin_layers)

        """ we use CE loss so we don't need to apply softmax (for test loss we also use the same CE. for accuracy
            we choose the highest value - this property is saved under softmax)"""

    def forward(self, x):
        if len(x.shape) == 3:
            x = x.view(1, *x.shape)
        assert x.shape[2] == x.shape[3] == self.params["in_wh"]
        assert x.shape[1] == self.params["channels_lst"][0]

        cnn_output = self.cnn(x).view((x.shape[0], -1))
        lin_output = self.linear_nn(cnn_output)
        return lin_output


def create_conv_nn(params):
    return ConvNN(params)


class LeNet5TrafficSign(nn.Module):
    name = "Lenet5-TrafficSign"
    def __init__(self, in_channels=3, out_channels=43):
        super().__init__()
        self.cnn = nn.Sequential(            
            nn.Conv2d(in_channels, 6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(6, 16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(16, 120, kernel_size=5, stride=1),
            nn.Tanh()
        )
        self.lin = nn.Sequential(
            nn.Linear(120, 84),
            nn.Tanh(),
            nn.Linear(84, 43)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = torch.flatten(x, 1)
        x = self.lin(x)
        return x


In [32]:
if run_experiment4 == True:
    inc_capacity_nets = []
    base_net_params = {
       "extras_blocks_components": [],
        # \"p_dropout\": 0.1,\n",
        "activation": torch.nn.ReLU,
        }

    base_net_params["out_size"] = 43
    base_net_params["in_wh"] = 32
    in_channels = 3


    for i in range(1,7):
        if  1<= i <= 7:
            base_net_params["channels_lst"] = [in_channels, 2 ** (i - 1), 2 ** (i + 1)]
            base_net_params["#FC_Layers"] = 1
            base_net_params["CNN_out_channels"] = 10 * i
            
        cap_net = create_conv_nn(base_net_params)
        inc_capacity_nets.append(cap_net)
        
    #pytorch_total_params = "parameters_{}".format(sum(p.numel() for p in net.parameters()))

    # run experiment 1 and 2 on each capacity
    for i, net in enumerate(inc_capacity_nets):
        net = net.to(device)
        net_name = "capacity_{}".format(i + 1)

In [26]:
net 

ConvNN(
  (cnn): Sequential(
    (0): Conv2d(3, 4, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): ReLU()
    (2): Conv2d(4, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): ReLU()
    (4): Conv2d(16, 30, kernel_size=(1, 1), stride=(1, 1))
    (5): ReLU()
  )
  (linear_nn): Sequential(
    (0): Linear(in_features=1920, out_features=43, bias=True)
  )
)

In [21]:
print(net_name)
print(summary(net, (3,32,32)))

capacity_1
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 1, 16, 16]              48
              ReLU-2            [-1, 1, 16, 16]               0
            Conv2d-3              [-1, 4, 8, 8]              64
              ReLU-4              [-1, 4, 8, 8]               0
            Conv2d-5             [-1, 10, 8, 8]              50
              ReLU-6             [-1, 10, 8, 8]               0
            Linear-7                   [-1, 43]          27,563
Total params: 27,725
Trainable params: 27,725
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.02
Params size (MB): 0.11
Estimated Total Size (MB): 0.14
----------------------------------------------------------------
None


In [24]:
print(net_name)
print(summary(net, (3,32,32)))

capacity_2
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 2, 16, 16]              96
              ReLU-2            [-1, 2, 16, 16]               0
            Conv2d-3              [-1, 8, 8, 8]             256
              ReLU-4              [-1, 8, 8, 8]               0
            Conv2d-5             [-1, 20, 8, 8]             180
              ReLU-6             [-1, 20, 8, 8]               0
            Linear-7                   [-1, 43]          55,083
Total params: 55,615
Trainable params: 55,615
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.04
Params size (MB): 0.21
Estimated Total Size (MB): 0.26
----------------------------------------------------------------
None


In [27]:
print(net_name)
print(summary(net, (3,32,32)))

capacity_3
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 16, 16]             192
              ReLU-2            [-1, 4, 16, 16]               0
            Conv2d-3             [-1, 16, 8, 8]           1,024
              ReLU-4             [-1, 16, 8, 8]               0
            Conv2d-5             [-1, 30, 8, 8]             510
              ReLU-6             [-1, 30, 8, 8]               0
            Linear-7                   [-1, 43]          82,603
Total params: 84,329
Trainable params: 84,329
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.06
Params size (MB): 0.32
Estimated Total Size (MB): 0.39
----------------------------------------------------------------
None


In [29]:
print(net_name)
print(summary(net, (3,32,32)))

capacity_4
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 16, 16]             384
              ReLU-2            [-1, 8, 16, 16]               0
            Conv2d-3             [-1, 32, 8, 8]           4,096
              ReLU-4             [-1, 32, 8, 8]               0
            Conv2d-5             [-1, 40, 8, 8]           1,320
              ReLU-6             [-1, 40, 8, 8]               0
            Linear-7                   [-1, 43]         110,123
Total params: 115,923
Trainable params: 115,923
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.10
Params size (MB): 0.44
Estimated Total Size (MB): 0.56
----------------------------------------------------------------
None


In [31]:
print(net_name)
print(summary(net, (3,32,32)))

capacity_5
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 16, 16]             768
              ReLU-2           [-1, 16, 16, 16]               0
            Conv2d-3             [-1, 64, 8, 8]          16,384
              ReLU-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 50, 8, 8]           3,250
              ReLU-6             [-1, 50, 8, 8]               0
            Linear-7                   [-1, 43]         137,643
Total params: 158,045
Trainable params: 158,045
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.17
Params size (MB): 0.60
Estimated Total Size (MB): 0.79
----------------------------------------------------------------
None


In [33]:
print(net_name)
print(summary(net, (3,32,32)))

capacity_6
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 16, 16]           1,536
              ReLU-2           [-1, 32, 16, 16]               0
            Conv2d-3            [-1, 128, 8, 8]          65,536
              ReLU-4            [-1, 128, 8, 8]               0
            Conv2d-5             [-1, 60, 8, 8]           7,740
              ReLU-6             [-1, 60, 8, 8]               0
            Linear-7                   [-1, 43]         165,163
Total params: 239,975
Trainable params: 239,975
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.31
Params size (MB): 0.92
Estimated Total Size (MB): 1.24
----------------------------------------------------------------
None


# Resnet

In [39]:
model = resnet50(pretrained=False)
model.fc = torch.nn.Linear(2048,43)
model.conv1 = torch.nn.Conv2d(3,64,kernel_size=5,stride=1)

#model = model.to('cpu')
model = model.to(device)
model 

/volumes1/thesis/notebooks/notebook_env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/volumes1/thesis/notebooks/notebook_env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1):

In [40]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

23591595

In [43]:
net = resnet101(weights = None )
net.fc = torch.nn.Linear(2048,43)
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [44]:
pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
pytorch_total_params

42588267

In [52]:
model = resnet152()
model.fc = torch.nn.Linear(2048,43)
model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [54]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

58231915